In [1]:
# Every logic run, make 10 frames with: Running logic=> macro-action inferred
# During macro-action keep ma timer going. Micro-steps, Macro-steps and global steps
# Display: SUCCESS or FAILURE
import os
import sys
# os.chdir('/Users/ludo/Desktop/animalai/animalai')
os.chdir('/Users/ludo/Desktop/animalai/animalai/neuro')
sys.path.insert(0, "/Users/ludo/Desktop/animalai/animalai/animalai_train")
sys.path.insert(1, "/Users/ludo/Desktop/animalai/animalai/animalai")

In [2]:
import json

from animalai.envs.gym.environment import AnimalAIGym
from animalai.envs.arena_config import ArenaConfig
from centroidtracker import CentroidTracker

from macro_action import MacroAction
from logic import Logic
from utils import preprocess, object_types
from config import COMPETITION_CONFIGS as comp
fpath = "/Users/ludo/Desktop/demo_vids/numerosity-fail/"

class Pipeline:
    def __init__(self, args):
        self.args = args
        self.ct = None
        self.gg_id = 0
        env_path = args.env
        worker_id = 1
        seed = args.seed
        self.arenas = args.arena_config
        self.buffer_size = 10
        first_arena = self.arenas[0] if self.arenas is not None else None

        # ac = ArenaConfig(arena_path)
        # Load Unity environment based on config file with Gym or ML agents wrapper
        self.env = AnimalAIGym(
            environment_filename=env_path,
            worker_id=worker_id,
            n_arenas=1,
            arenas_configurations=first_arena,
            seed=seed,
            grayscale=False,
            resolution=640,
            inference=args.inference
        )
        self.logic = Logic(self.buffer_size)

    def take_macro_step(self, env, state, step_results, macro_action, pass_mark):
        ma = MacroAction(env, self.ct, state, step_results, macro_action)
        # print(f"Initiating macro_action: {macro_action['initiate']}")
        step_results, state, macro_stats, micro_step = ma.run(pass_mark)
        # print(f"Results: {self.format_macro_results(macro_stats)}")
        return step_results, state, micro_step, macro_stats["success"]

    def episode_over(self, done):
        if done:
            return True
        return False

    def test_run(self, comp_fpath:str):
        try:
            for arena in comp['Numerosity'][:1]:
                print(f"ARENA:{arena}")
                ac = ArenaConfig(comp_fpath + arena + '.yml')
                # ac.arenas[0].t = 1000
                self.env.reset(ac)
                step_results = self.env.step([[0, 0]])  # Take 0,0 step
                global_steps = 0
                macro_step = 0
                reward = 0
                self.ct = {ot: CentroidTracker() for ot in object_types} # Initialise tracker
                actions_buffer = []
                while not self.episode_over(step_results[2]):
                    state = preprocess(self.ct, step_results, global_steps, reward)
                    macro_action = self.logic.run(
                        macro_step,
                        state,
                        choice="test")

                    step_results, state, micro_step, success = self.take_macro_step(
                        self.env, state, step_results, macro_action, ac.arenas[0].pass_mark
                    )
                    global_steps += micro_step
                    macro_step +=1
                    actions_buffer.append(macro_action['raw'][0])
                    if (state['reward']>ac.arenas[0].pass_mark):
                        break
                if (state['reward']>ac.arenas[0].pass_mark):
                    success = True
                else:
                    success = False
                with open(f"{fpath}mas.txt", "w") as text_file:
                    text_file.write(str(actions_buffer))
                
        except Exception as e:
            print(e)
        self.env.close()
        return success


In [3]:
from collections import namedtuple
from animalai.envs.arena_config import ArenaConfig
env_path = '../env/aaiagain'
ac = "configurations/curriculum4/35.yml"
ac = [ArenaConfig(ac)]
margs = namedtuple('args', 'seed env arena_config num_episodes inference')
args = margs(env=env_path, seed =2, arena_config=ac, num_episodes=120, inference=False)
pipe = Pipeline(args)
results = pipe.test_run("../competition_configurations/")


USING LOCAL LIBRARY


INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
INFO:mlagents_envs:Connected new brain:
AnimalAI?team=0
INFO:gym_unity:1 agents within environment.


ARENA:9-3-1
More on left


INFO:mlagents_envs:Environment shut down with return code 0.


In [ ]:
import cv2
import numpy as np
import glob
img_array = []
# fpath = "/Users/ludo/Desktop/demo_vids/y-maze-fail/"

file = open(f"{fpath}mas.txt", "r").read()
order = {}
for i in eval(file):
    order[int(
        i.split(',')[-1].replace(')',''))] = i.split('initiate(')[1].split('(')[0]

final_file_list = []
files = glob.glob(f'{fpath}*.png')
idxs = {}
c = 0
for k,v in order.items():
    tmp_list = [i for i in files if v in i]
    tmp_list = sorted(tmp_list, key=lambda x: int(x.split('/')[-1].split('-')[1].split('.')[0]))
    c+=len(tmp_list)
    idxs[v] = c
    final_file_list+=tmp_list
    
print(idxs)
for c, filename in enumerate(final_file_list):
    ma = [v for k,v in order.items() if v in filename][0]

    if c==0:
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        cv2.putText(img,f"UNDERSTANDING...", (int(width/10),int(height/10)), 2, 1, (255,255,255))
        for _ in range(23):
            img_array.append(img)
        img = cv2.imread(filename)
        cv2.putText(img,f"Initiating macro-action:", (int(width/10),int(height/10)), 0, 0.75, (255,255,255))
        cv2.putText(img,f"{ma} object 0", (int(width/10),int(height/10*2)), 0, 0.75, (255,255,255))
        for _ in range(15):
            img_array.append(img)
        idxs.pop(ma)
    if ma in idxs:
        img = cv2.imread(filename)
        if ma=='interact':
            cv2.putText(img,f"Goal visible=>", (int(width/10),int(height/10)), 0, 0.75, (255,255,255))
            cv2.putText(img,f"initiating macro-action:", (int(width/10),int(height/10*2)), 0, 0.75, (255,255,255))
            cv2.putText(img,f"{ma} w/ goal", (int(width/10),int(height/10*3)), 0, 0.75, (255,255,255))
            for _ in range(23):
                img_array.append(img)
        idxs.pop(ma)

    img = cv2.imread(filename)
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    cv2.putText(img,f"MACRO-ACTION: {ma}", (int(width/10),int(height/10)), 2, 1, (255,255,255))

    img_array.append(img)


out = cv2.VideoWriter(f'{fpath}/video.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 13, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()


In [10]:
import cv2
import numpy as np
import glob
img_array = []
# fpath = "/Users/ludo/Desktop/demo_vids/y-maze-fail/"

file = open(f"{fpath}mas.txt", "r").read()
order = {}
for i in eval(file):
    order[int(
        i.split(',')[-1].replace(')',''))] = i.split('initiate(')[1].split('(')[0]

final_file_list = []
files = glob.glob(f'{fpath}*.png')
idxs = {}
c = 0
for k,v in order.items():
    tmp_list = [i for i in files if v in i]
    tmp_list = sorted(tmp_list, key=lambda x: int(x.split('/')[-1].split('interact')[0]))
    c+=len(tmp_list)
    idxs[v] = c
    final_file_list+=tmp_list
    break
    
print(idxs)
for c, filename in enumerate(final_file_list):
    ma = [v for k,v in order.items() if v in filename][0]

    if c==0:
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        cv2.putText(img,f"UNDERSTANDING...", (int(width/10),int(height/10)), 2, 1, (255,255,255))
        for _ in range(23):
            img_array.append(img)
        img = cv2.imread(filename)
        cv2.putText(img,f"More goals on left", (int(width/10),int(height/10)), 0, 0.75, (255,255,255))
        cv2.putText(img,f"Selecting random goal on left id=0", (int(width/10),int(height/10*2)), 0, 0.75, (255,255,255))
        cv2.putText(img,f"Initiating macro-action:", (int(width/10),int(height/10*3)), 0, 0.75, (255,255,255))
        cv2.putText(img,f"{ma} w/goal id:0", (int(width/10),int(height/10*4)), 0, 0.75, (255,255,255))
        for _ in range(15):
            img_array.append(img)
        idxs.pop(ma)
    if ma in idxs:
        img = cv2.imread(filename)
        if ma=='interact':
            cv2.putText(img,f"Goal visible=>", (int(width/10),int(height/10)), 0, 0.75, (255,255,255))
            cv2.putText(img,f"initiating macro-action:", (int(width/10),int(height/10*2)), 0, 0.75, (255,255,255))
            cv2.putText(img,f"{ma} w/ goal", (int(width/10),int(height/10*3)), 0, 0.75, (255,255,255))
            for _ in range(23):
                img_array.append(img)
        idxs.pop(ma)

    img = cv2.imread(filename)
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    cv2.putText(img,f"MACRO-ACTION: {ma}", (int(width/10),int(height/10)), 2, 1, (255,255,255))

    img_array.append(img)


out = cv2.VideoWriter(f'{fpath}/video.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 13, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()


{'interact': 249}


In [ ]:
[filename for filename in glob.glob('/Users/ludo/Desktop/demo_vids/fc-suc/*.png')]

In [ ]:
os.listdir('/Users/ludo/Desktop/demo_vids/fc-suc/')

In [ ]:
file = open("/Users/ludo/Desktop/demo_vids/fc-suc/mas.txt", "r").read()
order = {}
for i in eval(file):
    order[int(
        i.split(',')[-1].replace(')',''))] = i.split('initiate(')[1].split('(')[0]
order
# int(eval(file)[1].split(',')[-1].replace(')',''))

In [ ]:
file = open("/Users/ludo/Desktop/demo_vids/fc-suc/mas.txt", "r").read()
order = {}
for i in eval(file):
    order[int(
        i.split(',')[-1].replace(')',''))] = i.split('initiate(')[1].split('(')[0]

final_file_list = []
files = glob.glob('/Users/ludo/Desktop/demo_vids/fc-suc/*.png')
for k,v in order.items():
    tmp_list = [i for i in files if v in i]
    tmp_list = sorted(tmp_list, key=lambda x: int(x.split('/')[-1].split('-')[1].split('.')[0]))
    final_file_list+=tmp_list
    print(len(final_file_list))
final_file_list

In [ ]:
import glob
files = glob.glob('/Users/ludo/Desktop/demo_vids/fc-suc/*.png')
# int(files[0].split('/')[-1].split('-')[1].split('.')[0])
files

In [ ]:
os.path.exists("/Users/ludo/Desktop/demo_vids/fc-suc/interact-1.png")